# Script XA DSA
This notebook imports necessary libraries and data from Excel files to work with ...

In [ ]:
import dt_ids7_export_module as bh_utils
import mapping_module as bh_map
import plot_module as bh_plot

# Get the name of the computer:
import socket
hostname = socket.gethostname()
if hostname == 'BHs-Desktop':
    path = 'C:\\' # 'E:\\'
elif hostname == 'BH-XPS':
    path = 'G:\\'
else:
    path = 'D:\\'


# Import the IDS7 data:
root_folder = path + "Users\\bjorn\\Downloads\\repdoser\\IDS7\\2023\\"
df_ids7 = bh_utils.import_excel_files_to_dataframe(root_folder)

# Import the IDS7 data:
root_folder = path + "Users\\bjorn\\Downloads\\repdoser\\DoseTrack - Serienivå\\2023\\"
df_dt = bh_utils.import_excel_files_to_dataframe(root_folder)

In [ ]:
# Run all cleanups, filters and checks:
df_ids7 = bh_utils.run_all_cleanup_filters_and_checks(df_ids7, df_dt, True)
# Merge the dataframes:
data = bh_utils.merge_ids7_dt(df_ids7, df_dt, True)

In [ ]:
# Print all unique elements in Modality Room, sorted alphabetically:
print("Unique elements in Modality Room:")
print(sorted(data['Modality Room'].unique()))

In [ ]:
import mapping_dict_rad_xa as rad_dict
# Representative doser for radiologisk Rikshospital
data = data[data['Modality Room'].isin(['IRH_XA7', 'RAK_XA1', 'RRA_XA1', 'RRH_RF1', 'RRH_RF2', 'RRH_XA1', 'RRH_XA2', 'RRH_XA4', 'RUL_RF1', 'RUL_RF2', 'RUL_RF3', 'RUL_XA3', 'RUL_XA5', 'RUL_XA6', 'RUL_XA7(Stue 15)'])]
import importlib
importlib.reload(rad_dict)


# Get the mapping dictionary:
mapping_dict = rad_dict.get_rad_xa_mapping_dict()
data = bh_map.map_procedures(data, mapping_dict, True)


In [ ]:
procedure = 'Caput Embolisering'
export = False
bh_plot.plot_representative_dose(data, procedure, 200, export)

In [ ]:
procedure = 'Trombektomi'
bh_plot.plot_representative_dose(data, procedure, 200, export)

In [ ]:
procedure = 'Caput og collum'
bh_plot.plot_representative_dose(data, procedure, 200, export)

In [ ]:
procedure = 'Myelografi'
bh_plot.plot_representative_dose(data, procedure, 175, export)

In [ ]:
procedure = 'Vertebroplastikk'
bh_plot.plot_representative_dose(data, procedure, 100, export)

In [ ]:
procedure = 'RG Columna eks. Myelografi og vertebroplastikk'
bh_plot.plot_representative_dose(data, procedure, 40, export)

In [ ]:
procedure = 'Cervical Nerverotsinjeksjon'
bh_plot.plot_representative_dose(data, procedure, 25, export)

In [ ]:
procedure = 'Cochlia'
bh_plot.plot_representative_dose(data, procedure, 10, export)

In [ ]:
procedure = 'Scoliose'
bh_plot.plot_representative_dose(data, procedure, 10, export)

In [ ]:
procedure = 'Shuntveier/Shuntventil'
bh_plot.plot_representative_dose(data, procedure, 35, export)

In [ ]:
procedure = 'Diafragmabevegelse'
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'HSG'
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Svelgfunksjon'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Øsofagus/ØVD, eks. svelgfunksjon'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Nedleggelse av ventrikkelsonde'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Urethragrafi/Urografi/Urinveier/MUCG'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Defecografi'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Gastroskopi'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Videofluoroskopi'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Nefrostomi innleggelse'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Nefrostomi/Pyelografi skifte eller fjerning'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'TACE'
export = False
bh_plot.plot_representative_dose(data, procedure, 200, export)

In [ ]:
procedure = 'PTC/PTBD/ERCP'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'RG Galleblære u. PTC eller Galleveier'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'BEVAR/FEVAR'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'TEVAR'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'EVAR'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Abdomen/Bekken/Aorta PTA/Embolisering/Stent/Stentgraft'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Milt/Nyrer'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'RG Abdomen - Abscessografi, Fjerning/Skifte av dren'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Pulmonalarterier/PTA/Embolisering'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'RGV Thorax'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Lymfangiografi'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Underex'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)

In [ ]:
procedure = 'Overex'
export = False
bh_plot.plot_representative_dose(data, procedure, 20, export)